# Evaluating RAG Systems with FloTorch

[FloTorch](https://www.flotorch.ai/) offers a robust evaluation framework for Retrieval-Augmented Generation (RAG) systems, enabling comprehensive assessment and comparison of Large Language Models (LLMs). It focuses on key metrics such as accuracy, cost, and latency, crucial for enterprise-level deployments.

## Key Evaluation Metrics for this Notebook

In this notebook, we will focus on evaluating our RAG pipelines using the following metrics:

* **Correctness:** This refers to the total number of samples that semantically both generated and expected are mateched

* **Inference Cost:** This refers to the total cost incurred for invoking Bedrock models to generate responses for all entries in the ground truth dataset.

* **Latency:** This measures the time taken for the inference process, specifically the duration of the Bedrock model invocations.


RAG systems are evaluated using a scoring method that measures response quality to questions in the evaluation set. Responses are rated as correct, Missing or incorrect:

- correct: The response correctly answers the user question and contains no hallucinated content.

- Missing: The answer does not provide the requested information. Such as “I don’t know”, “I’m sorry I can’t find …” or similar sentences without providing a concrete answer to the question.

- Incorrect: The response provides wrong or irrelevant information to answer the user question



### Load env variables

In [9]:
import json
with open("../Lab 1/variables.json", "r") as f:
    variables = json.load(f)

variables

{'accountNumber': '746074413210',
 'regionName': 'us-west-2',
 'collectionArn': 'arn:aws:aoss:us-west-2:746074413210:collection/3f35uv3lze9bdothrm0c',
 'collectionId': '3f35uv3lze9bdothrm0c',
 'vectorIndexName': 'ws-index-',
 'bedrockExecutionRoleArn': 'arn:aws:iam::746074413210:role/advanced-rag-workshop-bedrock_execution_role-us-west-2',
 's3Bucket': '746074413210-us-west-2-advanced-rag-workshop',
 'kbFixedChunk': 'WO4U6AWAU1',
 'kbSemanticChunk': 'OUFEWBGEES',
 'kbHierarchicalChunk': 'IHWIS6EP0H'}

### Evaluation Config

In [10]:
evaluation_config_data = {
   "eval_embedding_model" : "amazon.titan-embed-text-v2:0",
   "eval_retrieval_model" : "bedrock/cohere.command-r-v1:0",
   "eval_retrieval_service" : "bedrock",
   "aws_region" : variables['regionName'],
   "eval_embed_vector_dimension" : 1024
}

### Load RAG response data 

In [11]:
import json

filename = f"../results/ragas_evaluation_responses_for_different_models.json"

with open(filename, 'r', encoding='utf-8') as f:
    loaded_responses = json.load(f)


### Accuracy Evaluation with Custom Evaluation

In [12]:
from custom_evaluator import CustomEvaluator

In [13]:
processor = CustomEvaluator(evaluator_llm_info = evaluation_config_data)
evaluation_metrics = {}
for model_id, inference_data in loaded_responses.items():
    results = processor.evaluate(inference_data)
    evaluation_metrics[model_id] = results
    print(f"Evaluation completed for {model_id}")

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:28<00:00,  1.44s/it]


Evaluation completed for us.amazon.nova-micro-v1:0


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:28<00:00,  1.43s/it]


Evaluation completed for us.amazon.nova-pro-v1:0


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:35<00:00,  1.76s/it]


Evaluation completed for us.anthropic.claude-3-5-haiku-20241022-v1:0


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.48s/it]

Evaluation completed for us.anthropic.claude-3-5-sonnet-20241022-v2:0


### Evaluation output

In [14]:
final_evaluation = evaluator.evaluate_results(evaluation_metrics)

### Cost and Latency Evaluation

In [15]:
from cost_compute_utils import calculate_cost_and_latency_metrics

for model in loaded_responses:
    inference_data = loaded_responses[model]
    cost_and_latency_metrics = calculate_cost_and_latency_metrics(inference_data, model,
                evaluation_config_data["aws_region"])
    print(cost_and_latency_metrics)
    if model not in final_evaluation:
        # Insert - key doesn't exist yet
        final_evaluation[model] = cost_and_latency_metrics
    else:
        # Update - key already exists
        final_evaluation[model].update(cost_and_latency_metrics)

{'inference_cost': 0.0038132850000000005, 'average_inference_cost': 0.00019066425, 'latency': 19610.0, 'average_latency': 980.5, 'processed_items': 20}
{'inference_cost': 0.0855448, 'average_inference_cost': 0.00427724, 'latency': 23764.0, 'average_latency': 1188.2, 'processed_items': 20}
{'inference_cost': 0.02659136, 'average_inference_cost': 0.001329568, 'latency': 132016.0, 'average_latency': 6600.8, 'processed_items': 20}
{'inference_cost': 0.381432, 'average_inference_cost': 0.0190716, 'latency': 127302.0, 'average_latency': 6365.1, 'processed_items': 20}


### Evaluation metrics as pandas df

In [16]:
import pandas as pd

# Convert the nested dictionary to a DataFrame
evaluation_df = pd.DataFrame.from_dict(final_evaluation, orient='index')

# If you want the kb_type as a column instead of an index
evaluation_df = evaluation_df.reset_index().rename(columns={'index': 'model'})

evaluation_df

,model,number of samples correct,inference_cost,average_inference_cost,latency,average_latency,processed_items
0,us.amazon.nova-micro-v1:0,17,0.003813,0.000191,19610.0,980.5,20
1,us.amazon.nova-pro-v1:0,20,0.085545,0.004277,23764.0,1188.2,20
2,us.anthropic.claude-3-5-haiku-20241022-v1:0,17,0.026591,0.001330,132016.0,6600.8,20
3,us.anthropic.claude-3-5-sonnet-20241022-v2:0,15,0.381432,0.019072,127302.0,6365.1,20
